# Máltækni 2022, verkefni 4

B hluti leystur.

## a.

Í þessum hluta finnið þið ykkur þjálfaðan markara (part-of-speech-tagger).

Höldum áfram með vinum okkar í hrokum og hleypidómum. Færum þau samt öll í lágstafi.


In [1]:
from collections import Counter
import re
import spacy

nlp = spacy.load("en_core_web_sm")

with open("./data/pride-and-prejudice.txt", "rb") as input_file:
  pp_raw = input_file.read().decode('utf8')
  # Fjarlægja _ utanum orð (sjá a. ii.); færa í lágstafi
  pp = re.sub(r"\_(.*)\_", r"\1", pp_raw).lower()

### a. i.

Markið allan textann.

In [2]:
doc = nlp(pp)

### a. ii.

Finnið öll orðin í textanum sem hafa fleiri en eitt mark (t.d. getur sleep bæði verið nafnorð (noun) og sagnorð (verb). Skrifið fall sem skilar þeim öllum.

In [3]:
def find_more_than_one_mark(marks):
  """
  Find word lemmas with more than one mark, returns as list of dict, e.g.:
  """
  found = []
  for token in marks:
    seen = next((item for item in found if item["word"] == token.lemma_), None)
    if seen:
      if not token.pos_ in seen["tags"]:
        seen["tags"].append(token.pos_)
    else:
      found.append({ "word": token.lemma_, "tags": [token.pos_]})
  return [item for item in found if len(item["tags"]) > 1]

Í `pos` fáum við „The simple [UPOS](https://universaldependencies.org/u/pos/) part-of-speech tag“. Viljum ekki fá það sem er flokkað í „Other“:

* `PUNCT` punctuation.
* `SYM` symbol. A symbol is a word-like entity that differs from ordinary words by form, function, or both.
* `X` other. The tag X is used for words that for some reason cannot be assigned a real part-of-speech category.

Aukalega sá ég að `_` var að koma oft fyrir, þá vegna þess að það var notað til að tákna skáletrun, t.d. `_Her_`, eða hvað?. Fjarlægjum með regex í inntaki. En sjáum svo að það kemur ennþá? Harðkóðum fjarlægingu á því! Af hverju gerist þetta? Hver er rétta leiðin til að díla?
Síðan fara `.` að koma fyrir seinna í greiningu...

In [4]:
def clean_doc(doc, pos_skip_list = [], skip_lemmas = [], skip_stops = True):
  """
  "Clean" input doc by removing tags, stopwords and specific lemmas.
  """
  return [token for token in doc if not (token.pos_ in pos_skip_list or (skip_stops and token.is_stop) or token.lemma_ in skip_lemmas)]

skipped_pos = ['PUNCT', 'SYM', 'X', 'SPACE']
skipped_lemmas = ['_', '.']
clean = clean_doc(doc, skipped_pos, skipped_lemmas)

In [5]:
more_than_one_mark = find_more_than_one_mark(clean)

In [22]:
more_than_one_mark[:10]

[{'word': 'pride', 'tags': ['NOUN', 'PROPN', 'VERB']},
 {'word': 'prejudice', 'tags': ['NOUN', 'VERB']},
 {'word': 'jane', 'tags': ['NOUN', 'PROPN', 'ADJ', 'VERB']},
 {'word': 'single', 'tags': ['ADJ', 'VERB']},
 {'word': 'good', 'tags': ['ADJ', 'ADV', 'NOUN', 'PROPN']},
 {'word': 'want', 'tags': ['NOUN', 'VERB']},
 {'word': 'little', 'tags': ['ADJ', 'ADV', 'NOUN']},
 {'word': 'view', 'tags': ['NOUN', 'VERB']},
 {'word': 'mind', 'tags': ['NOUN', 'VERB']},
 {'word': 'dear', 'tags': ['ADJ', 'NOUN', 'PROPN', 'VERB']}]

### a. iii.

Skrifið fall sem skilar því orði sem hefur flest mismunandi mörk, ásamt öllum mörkunum sem tilheyra því. Dæmi: (minni, [nhen, lkenvm, feveþ]).

In [7]:
def words_with_most_marks(marks):
  longest = 0
  longest_words = []
  for mark in marks:
    current_length = len(mark["tags"])
    if current_length == longest:
      longest_words.append(mark)
    
    if current_length > longest:
      longest = current_length
      longest_words = [mark]

  return longest_words

In [8]:
words_with_most_marks(more_than_one_mark)[:100]

[{'word': 'half', 'tags': ['ADV', 'DET', 'ADJ', 'NOUN', 'PRON', 'NUM']}]

### a. iv.

Skilið 10 algengustu pörunum á forminu orð/mark og tíðni þeirra (t.d. (hundur, nafnorð, 300) úr textanum.

In [9]:
pair_counter = Counter([tuple((token.lemma_, token.pos_)) for token in clean])
[{ 'pair': element, 'count': count } for element, count in pair_counter.most_common()][:10]

[{'pair': ('mr', 'PROPN'), 'count': 785},
 {'pair': ('elizabeth', 'PROPN'), 'count': 621},
 {'pair': ('say', 'VERB'), 'count': 451},
 {'pair': ('know', 'VERB'), 'count': 388},
 {'pair': ('darcy', 'PROPN'), 'count': 369},
 {'pair': ('mrs', 'PROPN'), 'count': 344},
 {'pair': ('think', 'VERB'), 'count': 333},
 {'pair': ('sister', 'NOUN'), 'count': 294},
 {'pair': ('bingley', 'PROPN'), 'count': 278},
 {'pair': ('bennet', 'PROPN'), 'count': 256}]

### a. v.

Skrifið fall sem skilar öllum þrístæðum (trigrams) af mörkum í textanum, alveg óháð því hvaða orð fylgja þeim, og raðar þeim eftir því hversu oft þær birtast. Formið á að vera [mark1, mark2, mark3].

In [10]:
def ordered_ngram_tags(doc, n):
  just_tags = [item.pos_ for item in doc]
  ngrams = []

  for index in range(len(just_tags)):
    if index + n <= len(just_tags):
      ngrams.append(just_tags[index : index + n])

  countable = [tuple(item) for item in ngrams]
  counter = Counter(countable)

  return counter.most_common()

In [11]:
top_10_tags = ordered_ngram_tags(clean, 3)[:10]

top_10_tags

[(('NOUN', 'VERB', 'NOUN'), 2656),
 (('NOUN', 'NOUN', 'NOUN'), 2394),
 (('NOUN', 'NOUN', 'VERB'), 2299),
 (('VERB', 'NOUN', 'NOUN'), 2250),
 (('VERB', 'NOUN', 'VERB'), 1938),
 (('NOUN', 'ADJ', 'NOUN'), 1498),
 (('NOUN', 'VERB', 'VERB'), 1443),
 (('VERB', 'VERB', 'NOUN'), 1430),
 (('ADJ', 'NOUN', 'NOUN'), 1353),
 (('VERB', 'ADJ', 'NOUN'), 1319)]

### a. v. bónus

Finnið allar orðarunurnar úr textanum sem fylgja þeirri runu. Ef algengasta markaþrístæðan er t.d. [determiner, noun, verb] skilið þið öllum þrístæðunum í textanum sem fylgja því formi, t.a.m. [[a, man, says], [the, woman sees], [an, ape, walks]]

In [12]:
def find_word_sequence_by_tag_sequence(doc, tag_sequence):
  found = []

  """
  track the candidates on the form
  (words: [word_1, word_2, ... word_n], index: n-1)
  """
  candidates = []
  next_candidates = []

  for tag in doc:
    current_tag = tag.pos_
    current_word = tag.lemma_

    # advance any advanceable candiates
    for candidate in candidates:
      if current_tag == tag_sequence[candidate["index"]]:
        candidate["words"].append(current_word)
        candidate["index"] = candidate["index"] + 1

        # is it complete?
        if len(candidate["words"]) == len(tag_sequence):
          found.append(candidate["words"])
        else:
          next_candidates.append(candidate)
      else:
        pass
    
    # new candidate?
    if current_tag == current_tag:
      next_candidates.append({ "words": [current_word], "index": 0 })

    candidates = next_candidates
    next_candidates = []
  return found


In [13]:
find_word_sequence_by_tag_sequence(clean, top_10_tags[0][0])[:10]

[['view', 'man', 'enter'],
 ['neighbourhood', 'truth', 'fix'],
 ['fix', 'mind', 'surround'],
 ['surround', 'family', 'consider'],
 ['lady', 'day', 'hear'],
 ['netherfield', 'park', 'let'],
 ['bennet', 'answer', 'want'],
 ['tell', 'objection', 'hear'],
 ['invitation', 'dear', 'know'],
 ['say', 'netherfield', 'take']]

Áhugavert! Hvað ef við lengjum rununa?

In [14]:
ordered_ngram_tags(clean, 12)[:1]

[(('VERB',
   'ADJ',
   'NOUN',
   'VERB',
   'NOUN',
   'NOUN',
   'NOUN',
   'NOUN',
   'NOUN',
   'NOUN',
   'NOUN',
   'NOUN'),
  4)]

In [15]:
find_word_sequence_by_tag_sequence(clean, ordered_ngram_tags(clean, 12)[0][0])[:1]

[['feel',
  'persuade',
  'real',
  'confidence',
  'subsist',
  'disappointment',
  'charlotte',
  'turn',
  'fonder',
  'regard',
  'sister',
  'rectitude']]

Ekki mikið úr þessu að fá, eða hvað? Betra að halda sig við minna N?

### a. vi.

Reynið að rugla markarann. Látið hann marka setningar sem þið teljið að gætu ruglað hann. Finnið a.m.k. þrjú dæmi um villur sem hann gerir og útskýrið hvað það er sem gæti hafa ruglað hann.

Þetta reynir á málfræði kunnáttu mína! Nýti mér listann að ofan fyrir orð sem hafa margar myndir, einnig nýtast [Universal POS tags](https://universaldependencies.org/u/pos/) og [Word Type](https://wordtype.org/) vel.

In [16]:
[(token.lemma_, token.pos_) for token in nlp("Dear john letter.")]

[('Dear', 'PROPN'), ('john', 'PROPN'), ('letter', 'NOUN'), ('.', 'PUNCT')]

Hér ætti `dear` að vera `ADJ`, ekki `PROPN`, `john` er réttilega `PROPN` en `letter` ætti að vera `NOUN`.
Þar sem þetta er frasi sem þarfnast lærdóms er þetta ekki markað rétt. Ástæða fyrir villu er að túlka þetta í heildina sem nafn?

Smá svindl því ef við stækkum setningu í að innihalda meira samhengi er þetta flokkað rétt 🙈.

In [17]:
[(token.lemma_, token.pos_) for token in nlp("A dear john letter.")]

[('a', 'DET'),
 ('dear', 'ADJ'),
 ('john', 'PROPN'),
 ('letter', 'NOUN'),
 ('.', 'PUNCT')]

In [18]:
[(token.lemma_, token.pos_) for token in nlp("Man, what a day this has been!")]

[('Man', 'PROPN'),
 (',', 'PUNCT'),
 ('what', 'PRON'),
 ('a', 'DET'),
 ('day', 'NOUN'),
 ('this', 'PRON'),
 ('have', 'AUX'),
 ('be', 'AUX'),
 ('!', 'PUNCT')]

`man` ranglega flokkað sem `PROPN` en ætti að vera `INTJ`—interjection—þar sem það er notað sem upphrópun!

In [19]:
[(token.lemma_, token.pos_) for token in nlp("The man eats, then shoots and leaves.")]

[('the', 'DET'),
 ('man', 'NOUN'),
 ('eat', 'VERB'),
 (',', 'PUNCT'),
 ('then', 'ADV'),
 ('shoot', 'NOUN'),
 ('and', 'CCONJ'),
 ('leave', 'NOUN'),
 ('.', 'PUNCT')]

Stolið frá bókinni „Eats, Shoots & Leaves“ þar sem við viljum endilega koma því fram að maðurinn borði, skjóti og fari síðan. `shoots` og `leaves` ættu að vera `VERB`, ekki `NOUN`.

## b.

Í þessum hluta finnið þið ykkur þjálfaðan þáttara (constituency parser).

### b. i.

Þáttið allan textann.

Notum [Berkeley Neural Parser, `benepar`](https://spacy.io/universe/project/self-attentive-parser).

In [20]:
import benepar

# Sækjum gögn fyrir þáttarann, keyrum einu sinni
# benepar.download('benepar_en3')

# Notum stærra módel fyrir ensku
nlpx = spacy.load('en_core_web_md')

nlpx.add_pipe('benepar', config={'model': 'benepar_en3'})

Hér virkaði ekki að keyra neinn kóða gegnum Visual Studio Code, en keyrði í jupyter?

`TypeError: Couldn't build proto file into descriptor pool: duplicate file name (sentencepiece_model.proto)`

Aukalega verður allt mun hægara hér.. takmörkum okkur við 20þ orð.

In [114]:
pp_limited = " ".join(pp.split("\n")[:20000])
cp_parsed = nlpx(pp_limited)

### b. ii.

Skrifið fall sem skilar öllum nafnliðum (noun phrases) í textanum og raðar þeim eftir því hversu oft þeir birtast.

In [64]:
def ordered_noun_phrases(noun_phrases):
    counter = Counter([phrase.text for phrase in noun_phrases])
    return [{ "noun_phrase": element, 'count': count } for element, count in counter.most_common()]

In [115]:
ordered_noun_phrases(cp_parsed.noun_chunks)[:20]

[{'noun_phrase': 'i', 'count': 2062},
 {'noun_phrase': 'she', 'count': 1705},
 {'noun_phrase': 'it', 'count': 1531},
 {'noun_phrase': 'you', 'count': 1356},
 {'noun_phrase': 'he', 'count': 1332},
 {'noun_phrase': 'her', 'count': 766},
 {'noun_phrase': 'him', 'count': 761},
 {'noun_phrase': 'they', 'count': 601},
 {'noun_phrase': 'elizabeth', 'count': 529},
 {'noun_phrase': 'which', 'count': 522},
 {'noun_phrase': 'me', 'count': 445},
 {'noun_phrase': 'them', 'count': 435},
 {'noun_phrase': 'what', 'count': 378},
 {'noun_phrase': 'who', 'count': 285},
 {'noun_phrase': 'that', 'count': 276},
 {'noun_phrase': 'we', 'count': 252},
 {'noun_phrase': 'mr. darcy', 'count': 229},
 {'noun_phrase': 'jane', 'count': 222},
 {'noun_phrase': 'herself', 'count': 209},
 {'noun_phrase': 'nothing', 'count': 171}]

### b. iii.

Endurtakið ii. en sækið nú sagnliði (verb phrases).

Hér er ég aðeins farinn að taka flýtleiðir, næsta kemur að miklu leiti frá [stackoverflow svari](https://stackoverflow.com/questions/47856247/extract-verb-phrases-using-spacy).

In [89]:
from spacy.matcher import Matcher
from spacy.util import filter_spans

def parse_verb_phrases(doc):
    # Bad function, relies on global state
    patterns = [{'POS': 'VERB', 'OP': '?'},
               {'POS': 'ADV', 'OP': '*'},
               {'POS': 'AUX', 'OP': '*'},
               {'POS': 'VERB', 'OP': '+'}]

    matcher = Matcher(nlpx.vocab)
    matcher.add("Verb phrase", [pattern])

    matches = matcher(doc)
    spans = [doc[start:end] for _, start, end in matches]
    
    return spans

def ordered_verb_phrases(verb_phrases):
    counter = Counter([phrase.text for phrase in verb_phrases])
    return [{ "verb_phrase": element, 'count': count } for element, count in counter.most_common()]

ordered_verb_phrases(parse_verb_phrases(cp_parsed))[:10]

[{'verb_phrase': 'said', 'count': 32},
 {'verb_phrase': 'had', 'count': 25},
 {'verb_phrase': 'know', 'count': 24},
 {'verb_phrase': 'have', 'count': 20},
 {'verb_phrase': 'miss', 'count': 15},
 {'verb_phrase': 'think', 'count': 14},
 {'verb_phrase': 'see', 'count': 13},
 {'verb_phrase': 'do', 'count': 12},
 {'verb_phrase': 'dance', 'count': 11},
 {'verb_phrase': 'say', 'count': 10}]

### b. iv.

Reynið að rugla þáttarann. Látið hann þátta setningar sem þið teljið að gætu ruglað hann. Finnið a.m.k. þrjú dæmi um villur sem hann gerir og útskýrið hvað það er sem gæti hafa ruglað hann.

In [97]:
for chunk in nlpx("Table that for later").noun_chunks:
    print(chunk)
parse_verb_phrases(nlpx("Table that for later"))

[]

Hér ætti table að vera flokkað sem sögn en er flokkað sem hvorugt.

In [113]:
for chunk in nlpx("Let me Google that for you").noun_chunks:
    print(chunk)
parse_verb_phrases(nlpx("Let me Google that for you"))

me
you


[Let]

Svindl. Hér ætti Google að vera sögn (og er flokkuð sem slík ef við setjum G í lágstaf)

Að finna þessi dæmi var erfiðasti parturinn við verkefnið.